# Agenda

1. Iterator protocol
    - Adding iterations to classes
    - Generator functions
    - Generator comprehensions
2. Decorators
3. Threading and multiprocessing

# `__str__` vs. `__repr__`

Both of these methods should return strings. The question is, when is each one run?

Normally `__str__` is run:
- Whenever we run `str` on something
- If we use `print` on something (because it uses `str` behind the scenes)
- In other words: When we want to turn our object into a string, to display it to end users

Normally, `__repr__` is run:
- If we're in a debugger
- If we're in Jupyter (and not using `print`, but just asking for the value of a variable)
- Inside of other data structures (e.g., if we have a list of `Scoop` objects, Python will use `__repr__` and not `__str__` to show us the list)
- In other words: It's meant for developers, not for end users

But:
- If we only define `__str__`, then it does *not* cover cases for `__repr__`
- If we only define `__repr__`, then it *DOES* cover all cases, including those of `__str__`
- In theory, `__repr__` is supposed to return a string that is a valid Python expression.

In [2]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'str for Person {vars(self)}'

    def __repr__(self):
        return f'repr for Person {vars(self)}'

    def __format__(self, code):
        return f'format for Person {vars(self)}'
    
p = Person('Reuven')

print(f'{p}')

repr for Person {'name': 'Reuven'}


# Iterator protocol




In [5]:
for one_character in 'abcd':
    print(one_character)

a
b
c
d


In [6]:
for index, one_character in enumerate('abcd'):
    print(f'{index}: {one_character}')

0: a
1: b
2: c
3: d


# What happens in a `for` loop?

- "Iterable" means: Can be put in a `for` loop
- "Iterator" means: We can ask it for its next object, if it has one
Sometimes, an iterable is an iterator, and sometimes it isn't.

- `for` turns to the object and asks if it's iterable. It does this by running the `iter` function on that object. (You will probably never run `iter` in an actual program.)
    - if `iter` returns an object, that object is the "iterator" on which we're going to work
    - Otherwise, we get a `TypeError` exception, indicating that no, the object is not iterable
- Assuming that we get an iterator object back, we call the `next` function on it.
    - We might get something back. That's the value for the current iteration
    - Or we might get `StopIteration`, an exception that says we're at the end of the iterator.
- We keep repeating the above section, calling `next` until we get a `StopIteration` exception.

In [7]:
s = 'abcd'

i = iter(s)   # we ask: is s iterable? If so, return its iterator and assign to i.

In [8]:
i

In [9]:
i = iter(s)
i

In [10]:
next(i)   # ask i for its next value

'a'

In [11]:
 next(i)

'b'

In [12]:
next(i)

'c'

In [13]:
next(i)

'd'

In [14]:
next(i)

StopIteration: 

In [20]:
# Let's create an iterable class!

class MyData:
    def __init__(self, data):
        print(f'\tNow in MyData.__init__')
        self.data = data
        self.index = 0
        
    def __iter__(self):   # this is called each time we *start* a loop on our object -- it returns the iterator
        print('\tNow in MyData.__iter__')
        return self       # meaning: who is my iterator? I'm my own iterator!
    
    def __next__(self):   # this is called once for each iteration
        print(f'\tNow in MyData.__next__: {vars(self)}')
        if self.index >= len(self.data):
            print(f'\tRaising StopIteration')
            raise StopIteration   # no message is needed, because the "for" loop won't read it
            
        value = self.data[self.index]   # get the current value
        self.index += 1                 # increment the counter
        print(f'\tReturning {value}')
        return value                    # return the value for this iteration

m = MyData('abcd')  

for one_item in m:
    print(one_item)

	Now in MyData.__init__
	Now in MyData.__iter__
	Now in MyData.__next__: {'data': 'abcd', 'index': 0}
	Returning a
a
	Now in MyData.__next__: {'data': 'abcd', 'index': 1}
	Returning b
b
	Now in MyData.__next__: {'data': 'abcd', 'index': 2}
	Returning c
c
	Now in MyData.__next__: {'data': 'abcd', 'index': 3}
	Returning d
d
	Now in MyData.__next__: {'data': 'abcd', 'index': 4}
	Raising StopIteration
